In [3]:
### Libraries ###

from bs4 import BeautifulSoup
import requests
import re

In [11]:
### Get monster list webpage html text ###

response = requests.get("https://www.dandwiki.com/wiki/5e_Monsters")

if response.status_code == 200:                                                 # Checks that the connection to the webpage was established
    monster_list_soup = BeautifulSoup(response.content, "html.parser")          # Makes the HTML soup from the content of the response and assigns it to a variable
else:
    print(response)                                                             # If not a good connection, prints response code

In [12]:
### Put monster page hyperlinks into list from monster_list_soup ###

monster_href = []                                                                       # List to store each monster's hyperlink

for a in monster_list_soup.find_all("a", href=re.compile("\\(5e_Creature\\)")):         # Finds all <a> tags that have a hyperlink to a monster. The '\' allows the parenthesis to be recognized
    monster_href.append("https://dandwiki.com" + a["href"])                             # Adds the web address prefix to the href attribute and appends it to the monster_href list

monster_href

['https://dandwiki.com/wiki/Sea_Serpent_(5e_Creature)',
 'https://dandwiki.com/wiki/Young_Sea_Serpent,_Variant_(5e_Creature)',
 'https://dandwiki.com/wiki/Aaeaea_(5e_Creature)',
 'https://dandwiki.com/wiki/Aalar%27s_Chosen_Flesh_Golem_(5e_Creature)',
 'https://dandwiki.com/wiki/Aalar%27s_Chosen_Grimlock_(5e_Creature)',
 'https://dandwiki.com/wiki/Aarakocra_Galewing_(5e_Creature)',
 'https://dandwiki.com/wiki/Aarakocra_Skeleton_(5e_Creature)',
 'https://dandwiki.com/wiki/Aarakocra_Sky_Lord_(5e_Creature)',
 'https://dandwiki.com/wiki/Abaddon_(5e_Creature)',
 'https://dandwiki.com/wiki/Abaddon_the_Despoiler_(5e_Creature)',
 'https://dandwiki.com/wiki/Abaddon%27s_Locusts_(5e_Creature)',
 'https://dandwiki.com/wiki/Abathrael_(5e_Creature)',
 'https://dandwiki.com/wiki/Aberrant_Demagogue_(5e_Creature)',
 'https://dandwiki.com/wiki/Abigor_(5e_Creature)',
 'https://dandwiki.com/wiki/Abominable_Yeti_Mage_(5e_Creature)',
 'https://dandwiki.com/wiki/Abomination_of_Flesh_(5e_Creature)',
 'https://

In [46]:
### Get webpage html text ###

response = requests.get(monster_href[0])

if response.status_code == 200:                                                   # Checks that the connection to the webpage was established
    soup = BeautifulSoup(response.content, "html.parser")                         # Makes the HTML soup from the content of the response and assigns it to variable MonsterName_soup
else:
    print(response)                                                               # If not a good connection, prints response code

monster_name = soup.find("span", class_="mw-headline").get_text()                 # The monsters name is in the first span of the page with class "mw-headline". This grabs the text and assigns it

size, body_type, alignment = soup.find("table").find("i").get_text().split()      # The size, body type, and alignment are all together in the first italic tag of the first table. This grabs and assgins the values
body_type = body_type.rstrip(",")                                                 # Removes a comma between body type and alignment

In [31]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Sea Serpent (5e Creature) - D&amp;D Wiki
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"dc50ac84d0da39ee12b677da","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Sea_Serpent_(5e_Creature)","wgTitle":"Sea Serpent (5e Creature)","wgCurRevisionId":1801233,"wgRevisionId":1801233,"wgArticleId":125583,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Featured Article","Completed Pages","CR18","5e","Creature","Monstrosity Type","Gargantuan Size","Underwater Monster